In [ ]:
! pip install optuna seqeval evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import ast
from tqdm import tqdm
from collections import Counter
import datasets
from datasets import (Dataset, Features, Sequence, Value, ClassLabel, load_dataset,
                    load_from_disk, concatenate_datasets, DatasetDict)
from sklearn.model_selection import KFold
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                         pipeline, TrainingArguments, Trainer,
                         DataCollatorForTokenClassification, EarlyStoppingCallback)
import torch
import optuna
import os
os.environ['WANDB_DISABLED'] = 'true'
import pickle
import numpy as np
import seqeval
import evaluate
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

seqeval = evaluate.load("seqeval")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
TRAIN_PATH = "/content/drive/MyDrive/Hackatons/X5_ner_2025/train.csv"
SUBMISSION_PATH = "/content/drive/MyDrive/Hackatons/X5_ner_2025/submission.csv"

## Подготовка датасета

In [ ]:
train_df_raw = pd.read_csv(TRAIN_PATH, sep=';')
train_df_raw

,sample,annotation
0,aa,"[(0, 2, 'O')]"
1,aala,"[(0, 4, 'O')]"
2,aarcca,"[(0, 6, 'O')]"
3,abon,"[(0, 4, 'O')]"
4,abso,"[(0, 4, 'B-BRAND')]"
...,...,...
27246,№1 кофейни,"[(0, 2, 'B-BRAND'), (3, 10, 'B-TYPE')]"
27247,№1 кофейник,"[(0, 2, 'B-BRAND'), (3, 11, 'B-TYPE')]"
27248,№1 пиво,"[(0, 2, 'B-BRAND'), (3, 7, 'B-TYPE')]"
27249,№1 са,"[(0, 2, 'B-BRAND'), (3, 5, 'B-TYPE')]"


In [ ]:
train_df_raw['annotation'] = train_df_raw['annotation'].str.replace("\'0\'", "O")

In [ ]:
train_df_raw['annotation'] = train_df_raw['annotation'].apply(lambda x: ast.literal_eval(str(x)))

In [ ]:
all_words = []
all_tags = []
for i, row in tqdm(train_df_raw.iterrows()):
    words_sample = []
    entities_sample = []
    for ent in row['annotation']:
        word = row['sample'][ent[0]:ent[1]]
        words_sample.append(word)
        entities_sample.append(ent[2])
    all_words.append(words_sample)
    all_tags.append(entities_sample)
    # print(words_sample, entities_sample)

27251it [00:02, 11506.34it/s]


In [ ]:
assert len(all_words) == len(all_tags), 'different lengths'

In [ ]:
lbls_in_dataset = [
 'O',
 'B-BRAND',
 'B-PERCENT',
 'B-TYPE',
 'B-VOLUME',
 'I-BRAND',
 'I-PERCENT',
 'I-TYPE',
 'I-VOLUME']
label2id = {v:i for i, v in enumerate(lbls_in_dataset)}
id2label = {i:v for i, v in enumerate(lbls_in_dataset)}

In [ ]:
features=Features(
    {
        "id": Value(dtype='int32', id=None),
        "tokens": Sequence(feature=Value(dtype='string', id=None)),
        "ner_tags": Sequence(feature=ClassLabel(num_classes=len(lbls_in_dataset), names=list(lbls_in_dataset)), id=None)
    }
)

In [ ]:
ds = Dataset.from_dict({"id": list(range(len(all_words))),
                        "tokens": all_words,
                        "ner_tags": all_tags},
                       features=features)

In [ ]:
ds_splitted = ds.train_test_split(
    test_size=0.25, shuffle=True, seed=42,
    # stratify_by_column='ner_tags'
)

In [ ]:
Counter([x for l in ds_splitted['train']['ner_tags'] for x in l])

Counter({3: 18399,
         7: 3399,
         0: 4022,
         1: 5438,
         5: 365,
         2: 17,
         4: 39,
         8: 19,
         6: 2})

In [ ]:
Counter([x for l in ds_splitted['test']['ner_tags'] for x in l])

Counter({3: 6129, 1: 1774, 7: 1133, 0: 1358, 5: 122, 2: 9, 4: 18, 8: 8, 6: 2})

## Загрузка модели

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ai-forever/ruT5-large',
                                          use_fast=True,
                                          add_prefix_space=True)

model = AutoModelForTokenClassification.from_pretrained('ai-forever/ruT5-large',
                                                        num_labels=len(lbls_in_dataset),
                                                        id2label=id2label,
                                                        label2id=label2id).to("cuda:0")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

Some weights of T5ForTokenClassification were not initialized from the model checkpoint at ai-forever/ruT5-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
s = '''
сироп топинамбура
'''
r = tokenizer(s)
[tokenizer.decode(x) for x in r.input_ids][:5]

['сироп', 'то', 'пина', 'м', 'бур']

## Токенизация и подготовка

In [ ]:
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_index = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [ ]:
tokenized_ds_train = ds_splitted['train'].map(tokenize_and_align_labels,
                                              batched=True,
                                              fn_kwargs={'tokenizer': tokenizer})

Map:   0%|          | 0/20438 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
tokenized_ds_test = ds_splitted['test'].map(tokenize_and_align_labels,
                                              batched=True,
                                              fn_kwargs={'tokenizer': tokenizer})

Map:   0%|          | 0/6813 [00:00<?, ? examples/s]

## Обучение

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

print("Compiling model for faster training...")
torch.set_float32_matmul_precision('high')
model.to(device)

Using device: cuda:0
Compiling model for faster training...


T5ForTokenClassification(
  (transformer): T5EncoderModel(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
                (wo):

In [ ]:
# функция метрики
def compute_metrics_custom(p):
    labels_list = list(id2label.values())
    predictions, labels = p
    if not os.path.exists('/content/test/p_trainer.pkl'):
        os.makedirs('/content/test', exist_ok=True)
        with open('/content/test/p_trainer.pkl', 'wb') as f:
            pickle.dump(p, f)
    predictions = np.argmax(predictions, axis=2)


    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    report_dict = classification_report(true_labels, true_predictions, digits=4, output_dict=True)
    report = classification_report(true_labels, true_predictions, digits=4)
    macro_f1 = report_dict["macro avg"]["f1-score"]
    print("=== seqeval classification_report ===")
    print(report)
    CLASS_REPORT_PATH = '/content/logs/last_classification_report.txt'
    try:
        with open(CLASS_REPORT_PATH, "w", encoding="utf-8") as f:
            f.write(report)
    except Exception as e:
        print(f"Warning: failed to write classification report to {CLASS_REPORT_PATH}: {e}")

    return {
        "f1_macro": macro_f1,
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
import optuna.logging
optuna.logging.set_verbosity(optuna.logging.INFO)

best_f1 = 0

GDRIVE_DIR = '/content/drive/MyDrive/Hackatons/X5_ner_2025/ruT5_large_250925'

def printer(s):
    print('*'*150, end='\n\n')
    print(s, end='\n\n')
    print('*'*150, end='\n\n')

In [ ]:
early_stop_cb = EarlyStoppingCallback(early_stopping_patience=2)

In [ ]:
def objective(trial: optuna.Trial, model=model):
    global best_f1
    model = model
    model_name = 'ruT5_large_250925_optuna_v1'
    trial_check_dir = f"./checkpoints_trial"

    os.makedirs(trial_check_dir, exist_ok=True)
    args = TrainingArguments(
        # f"{model_name}-finetuned-ner",
        output_dir=trial_check_dir,
        overwrite_output_dir=True,
        eval_strategy = "epoch",
        torch_compile=True,
        # 64\
        per_device_train_batch_size=256,
        per_device_eval_batch_size=256,
        learning_rate=trial.suggest_loguniform('learning_rate', low=1e-5, high=5e-4),
        weight_decay=trial.suggest_loguniform('weight_decay', 1e-4, 0.05),
        num_train_epochs=trial.suggest_int('num_train_epochs', low = 3, high = 10),
        seed=42,
        data_seed=24,
        gradient_accumulation_steps=2,
        warmup_ratio=0.1,
        report_to=None,
        logging_dir="./logs",
        logging_steps=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1_macro",
        # greater_is_better=False,
        save_total_limit=1,
        save_strategy="epoch",  # Changed to match evaluation_strategy
    )

    # early_stopping = EarlyStoppingCallback(
    #     early_stopping_patience=1,  # Stop if F1 decreases for 1 consecutive epoch
    #     early_stopping_threshold=0.001
    # )

    trainer = Trainer(
        model,
        args,
        train_dataset=tokenized_ds_train,
        eval_dataset=tokenized_ds_test,
        # train_dataset=small_dataset_train,
        # eval_dataset=small_dataset_test,
        data_collator=data_collator,
        compute_metrics=compute_metrics_custom,
        tokenizer=tokenizer,
        callbacks=[early_stop_cb],
    )

    trainer.train()

    # Evaluate and save best model globally
    eval_metrics = trainer.evaluate()
    current_f1 = eval_metrics["eval_f1_macro"]

    if current_f1 > best_f1:
        best_f1 = current_f1
        trainer.save_model("./best_model")
        trainer.save_model(GDRIVE_DIR)
        printer(f"New best model saved with F1: {best_f1:.4f}")
    return current_f1

In [ ]:
study = optuna.create_study(study_name='test_optuna', direction='maximize')
study.optimize(func=objective, n_trials=5)

[I 2025-09-25 15:47:56,064] A new study created in memory with name: test_optuna
/tmp/ipython-input-992928850.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', low=1e-5, high=5e-4),
/tmp/ipython-input-992928850.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay=trial.suggest_loguniform('weight_decay', 1e-4, 0.05),
The speedups for torchdynamo mostly come with GPU Ampere or higher and which is not detected here.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_

Epoch,Training Loss,Validation Loss,F1 Macro,Precision,Recall,F1,Accuracy
1,0.742300,0.537095,0.400311,0.803884,0.892531,0.845891,0.843418
2,0.362400,0.358328,0.450653,0.905844,0.943039,0.924067,0.915001
3,0.158600,0.302577,0.579675,0.930021,0.954029,0.941872,0.930194
4,0.158400,0.325737,0.720133,0.916179,0.951784,0.933642,0.924071
5,0.033900,0.308182,0.823673,0.938384,0.958659,0.948413,0.937526
6,0.041600,0.379980,0.770415,0.939387,0.954543,0.946905,0.936732
7,0.036000,0.425532,0.874428,0.938848,0.960670,0.949634,0.938206
8,0.014800,0.478993,0.876659,0.943126,0.959313,0.951151,0.938773
9,0.004200,0.514241,0.876177,0.942816,0.958425,0.950557,0.938093


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.6379    0.7954    0.7080      5043
     PERCENT     0.0000    0.0000    0.0000        14
        TYPE     0.8637    0.9250    0.8933     16297
      VOLUME     0.0000    0.0000    0.0000        29

   micro avg     0.8039    0.8925    0.8459     21383
   macro avg     0.3754    0.4301    0.4003     21383
weighted avg     0.8087    0.8925    0.8478     21383



/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch,Training Loss,Validation Loss,F1 Macro,Precision,Recall,F1,Accuracy
1,0.009600,0.497267,0.858961,0.939611,0.957583,0.948512,0.936846
2,0.001700,0.623256,0.883508,0.939863,0.956741,0.948227,0.936392
3,0.000200,0.633704,0.879540,0.940540,0.958706,0.949536,0.937866


=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.9097    0.9014    0.9056      5043
     PERCENT     1.0000    0.6429    0.7826        14
        TYPE     0.9487    0.9756    0.9619     16297
      VOLUME     0.8148    0.7586    0.7857        29

   micro avg     0.9396    0.9576    0.9485     21383
   macro avg     0.9183    0.8196    0.8590     21383
weighted avg     0.9394    0.9576    0.9483     21383

=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.9054    0.9032    0.9043      5043
     PERCENT     1.0000    0.6429    0.7826        14
        TYPE     0.9504    0.9736    0.9619     16297
      VOLUME     0.8438    0.9310    0.8852        29

   micro avg     0.9399    0.9567    0.9482     21383
   macro avg     0.9249    0.8627    0.8835     21383
weighted avg     0.9397    0.9567    0.9481     21383

=== seqeval classification_report ===
              pr

There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight'].


[I 2025-09-25 17:14:40,644] Trial 4 finished with value: 0.8835080653340189 and parameters: {'learning_rate': 0.00012973534241957156, 'weight_decay': 0.0006306410098490446, 'num_train_epochs': 3}. Best is trial 2 with value: 0.8926324166140165.


=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.9054    0.9032    0.9043      5043
     PERCENT     1.0000    0.6429    0.7826        14
        TYPE     0.9504    0.9736    0.9619     16297
      VOLUME     0.8438    0.9310    0.8852        29

   micro avg     0.9399    0.9567    0.9482     21383
   macro avg     0.9249    0.8627    0.8835     21383
weighted avg     0.9397    0.9567    0.9481     21383



In [ ]:
best_trial = study.best_trial
printer(f"Final best F1: {best_trial.value}")
printer(f"Final params: {best_trial.params}")

******************************************************************************************************************************************************

Final best F1: 0.8926324166140165

******************************************************************************************************************************************************

******************************************************************************************************************************************************

Final params: {'learning_rate': 6.44944805049113e-05, 'weight_decay': 0.004066034568680406, 'num_train_epochs': 9}

******************************************************************************************************************************************************



## тест

In [ ]:
df_test = pd.read_csv('submission.csv', sep=';', usecols=['sample'])

In [ ]:
best_model_path = '/content/drive/MyDrive/Hackatons/X5_ner_2025/ruT5_large_250925'
final_tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=True, add_prefix_space=True)
final_model = AutoModelForTokenClassification.from_pretrained(best_model_path)


In [ ]:
final_model.to('cuda:0')

T5ForTokenClassification(
  (transformer): T5EncoderModel(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
                (wo):

In [ ]:
token_classifier = pipeline(
    "token-classification", model=final_model, aggregation_strategy="first", tokenizer=final_tokenizer
)
for s in samples:
    print(f'sample: {s}')
    res = token_classifier(s)
    for i, r in enumerate(res):
        # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
        print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '. Probs:  ' + str(round(r['score'], 4)))
    print('#'*40)

Device set to use cuda:0


NameError: name 'samples' is not defined

In [ ]:
s = 'очиститель для унитаза'
res = token_classifier(s)
for i, r in enumerate(res):
    # print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'])
    print('Entity: '+ r['entity_group'] + '   Word: ' + r['word'] + '. Probs:  ' + str(round(r['score'], 4)))
print(res)

Entity: TYPE   Word: очиститель. Probs:  0.9998
[{'entity_group': 'TYPE', 'score': np.float32(0.9997584), 'word': 'очиститель', 'start': 0, 'end': 10}]


In [ ]:
lbls_in_dataset = [
 'O',
 'B-BRAND',
 'B-PERCENT',
 'B-TYPE',
 'B-VOLUME',
 'I-BRAND',
 'I-PERCENT',
 'I-TYPE',
 'I-VOLUME']
label2id = {v:i for i, v in enumerate(lbls_in_dataset)}
id2label = {i:v for i, v in enumerate(lbls_in_dataset)}

In [ ]:
import torch
import torch.nn.functional as F
import re

def predict_all_entities(text: str, model, tokenizer, id2label, device=None):
    """
    Word-level inference with original character spans (start_idx, end_idx, ENTITY).
    Returns a list of tuples for each word (including 'O').
    """

    model.eval()
    if device is None:
        device = next(model.parameters()).device

    # --- find words and their char spans in original text ---
    words = []
    spans = []
    for match in re.finditer(r"\S+", text):
        words.append(match.group())
        spans.append(match.span())  # (start_idx, end_idx)

    # encode with word-level info
    enc = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True
    )

    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)
    word_ids = enc.word_ids(batch_index=0)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits[0]               # (seq_len, num_labels)
        probs = F.softmax(logits, dim=-1)        # (seq_len, num_labels)
    # print()
    # print(probs)
    results = []
    prev_word_idx = None
    # print(word_ids)
    for token_idx, word_idx in enumerate(word_ids):
        # print(token_idx, word_idx, spans, logits[token_idx])
        if word_idx is None:
            prev_word_idx = None
            continue

        # only take first subtoken per word
        if word_idx != prev_word_idx:
            label_id = int(torch.argmax(logits[token_idx]).cpu().numpy())
            label = id2label[label_id]

            start_idx, end_idx = spans[word_idx]
            results.append((start_idx, end_idx, label))

        prev_word_idx = word_idx

    return results


# -------------------------
# Example usage
# -------------------------
s = "сыр натура сливочный"

# Example id2label (replace with your mapping)
# id2label = {0: "O", 1: "B-TYPE", 2: "I-TYPE", ...}

# res = predict_all_entities(s, final_model, final_tokenizer, id2label)
# print(res)


NameError: name 'final_model' is not defined

In [ ]:
annotations = []
for s in tqdm(df_test['sample'].tolist()):
    r = predict_all_entities(s, final_model, final_tokenizer, id2label)
    annotations.append(r)

100%|██████████| 5000/5000 [01:59<00:00, 41.73it/s]


In [ ]:
df_test['annotation'] = annotations

In [ ]:
df_test.sample(20)

,sample,annotation
1501,куркум,"[(0, 6, B-TYPE)]"
2586,вялены,"[(0, 6, B-TYPE)]"
2653,exponetto,"[(0, 9, B-BRAND)]"
1055,моцарелла шарики,"[(0, 9, B-TYPE), (10, 16, I-TYPE)]"
705,галеты petra,"[(0, 6, B-TYPE), (7, 12, B-BRAND)]"
106,ванилик,"[(0, 7, B-TYPE)]"
589,нектар j,"[(0, 6, B-TYPE), (7, 8, B-BRAND)]"
2468,тнк,"[(0, 3, O)]"
2413,рыбный фарш,"[(0, 6, B-TYPE), (7, 11, I-TYPE)]"
1600,сгущенные молочн,"[(0, 9, B-TYPE), (10, 16, I-TYPE)]"


In [ ]:
df_test.to_csv('submission_ruT5_large_250925.csv', index=False, sep=';')

## k-fold

In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a list to hold each fold (train/val split)
folds = []

for train_idx, val_idx in kf.split(ds):
    train_split = ds.select(train_idx)
    val_split = ds.select(val_idx)
    folds.append(DatasetDict({"train": train_split, "validation": val_split}))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ai-forever/ruT5-large',
                                          use_fast=True,
                                          add_prefix_space=True)

model = AutoModelForTokenClassification.from_pretrained('ai-forever/ruT5-large',
                                                        num_labels=len(lbls_in_dataset),
                                                        id2label=id2label,
                                                        label2id=label2id).to("cuda:0")

In [ ]:
lbls_in_dataset = [
 'O',
 'B-BRAND',
 'B-PERCENT',
 'B-TYPE',
 'B-VOLUME',
 'I-BRAND',
 'I-PERCENT',
 'I-TYPE',
 'I-VOLUME']
label2id = {v:i for i, v in enumerate(lbls_in_dataset)}
id2label = {i:v for i, v in enumerate(lbls_in_dataset)}

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
Final params: {'learning_rate': 6.44944805049113e-05, 'weight_decay': 0.004066034568680406, 'num_train_epochs': 9}

In [ ]:
df_all_valid_docs = pd.DataFrame()

for fold in folds:
    print(f"Start training for fold {fold}")
    # Fixed hyperparameters (no Optuna suggestions)
    training_args = TrainingArguments(
        eval_strategy="no",  # No evaluation during training
        torch_compile=True,
        per_device_train_batch_size=256,
        learning_rate=6.44944805049113e-05,          # fixed value (was suggested by Optuna before)
        weight_decay=0.004066034568680406,
        num_train_epochs=9,          # fixed
        seed=42,
        data_seed=24,
        gradient_accumulation_steps=2,
        warmup_ratio=0.1,
        report_to=None,
        logging_dir="./logs",
        logging_steps=1,
        )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=fold['train'],
        data_collator=data_collator,
        tokenizer=tokenizer,

    )

    # Start training
    print("Starting training without evaluation...")
    trainer.train()
    trained_model = trainer.model

    # Optional: Save final model
    print(f"Training completed for fold {fold}")

    df_valid = fold['validation'].to_pandas()
    predicted_ner_tags = []
    for doc in df_valid['tokens'].tolist()
        predicted_doc_ner = predict_all_entities(s, trained_model, tokenizer, id2label)
        predicted_ner_tags.append(predicted_doc_ner)
    df_valid['predicted_ner_tags'] = predicted_ner_tags

    if df_all_valid_docs.empty:
        df_all_valid_docs = df_valid
    else:
        df_all_valid_docs = pd.concat([df_all_valid_docs, df_valid])

## где ошибка в трейне

In [ ]:
df_valid_final[df_valid_final.annotation.astype(str) != df_valid_final.ann.astype(str)].to_csv('comparison_valid.csv', index=False)

In [ ]:
folds[0]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 21800
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 5451
    })
})

In [ ]:
dd = pd.DataFrame()
dd.empty

True